# Stony Rise Insect Light-trapping 1992-2019

This notebook visualises data from a long-term insect light-trapping dataset published to Zenodo as a Darwin Core Archive including sample events and representative images of many of the recorded species.

See [Catches of numerous insect species in Rothamsted 160W light trap at Devonport, Tasmania, 1992-2019](https://doi.org/10.5281/zenodo.6793249). The data may also be [accessed through GBIF](https://www.gbif.org/dataset/044f96bc-3bf2-4a38-9f7c-8808ab48dbf1).

The dataset ZIP file includes the following files used in this notebook:

 * **event.csv** - Darwin Core sample event records for each period in which insects were collected in the trap
 * **taxon.csv** - Darwin Core taxon records for each species (or operational taxonomic unit) sampled and monitored for at least some part of the period 1992-2019, including identifiers for the first and last events in which the species was monitored and, where representative images exist for these taxa, paths to the image files inside the ZIP file
 * **occurrence.csv** - Darwin Core occurrence records indicating presence/absence and counts of individuals for each taxon in each event in which it was monitored
 * **image/** - Folder containing image files referenced in taxon.csv

### Initialization

Import libraries, define key constants, prepare option widgets.

In [33]:
import io
import os
import sys
import csv
import requests
import ipywidgets as widgets
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import PIL
import shutil
import statsmodels.api as sm

%matplotlib inline

zipfile_name = "StonyRiseLightTrappingDataset.zip"
download_url = "https://zenodo.org/record/6820319/files/" + zipfile_name + "?download=1"
local_folder = "StonyRiseLightTrappingDataset"

start_year = 1992
end_year = 2019
num_years = end_year - start_year + 1

year_labels = [str(i) for i in range(start_year, end_year + 1)]
month_labels = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

force_data_download = widgets.Checkbox(value = False, description = "Force data download")

### Options

This notebook will download and unpack the dataset from Zenodo if a local copy is not found. The *Force data download* option overrides this vehaviour and forces download.

In [34]:
display(force_data_download)

Checkbox(value=False, description='Force data download')

### Download

Download and unzip data from Zenodo if not already present or *Force data download* specified.

In [6]:
if not os.path.exists(local_folder) or force_data_download.value:
    if os.path.exists(local_folder):
        shutil.rmtree(local_folder)
    print("Downloading from " + download_url + " - please wait ...")
    response = requests.get(download_url)
    open(zipfile_name, "wb").write(response.content)
    print("Unzipping " + zipfile_name)
    os.mkdir(local_folder)
    with zipfile.ZipFile(zipfile_name, 'r') as zip_ref:
        zip_ref.extractall(local_folder)
    print("Cleaning up")
    os.remove(zipfile_name)
    
print(local_folder + " downloaded from Zenodo")

Unzipping StonyRiseLightTrappingDataset.zip
Cleaning up
StonyRiseLightTrappingDataset downloaded from Zenodo


### Load

Load all records from taxon.csv and event.csv as lists of Python objects backed by dictionaries to find the index of taxon and event records by identifier.

Read all records from occurrence.csv and generate two tables:

 * **individuals** - The number of individuals of each taxon recorded in each calendar month of each year from 1992 to 2019
 * **nights** - The number of nights that each taxon was monitored in each calendar month of each year from 1992 to 2019
 
Most event records relate to single nights, but some events represent a combined sample from several consecutive nights, potentially spanning the end of one month and the start of the next. In these cases, the individuals and the nights are pro-rated across the two months concerned.

In [7]:
class Taxon:
  def __init__(self, row):
    self.ID = row[0]
    self.verbatimIdentification = row[1]
    self.order = row[2]
    self.family = row[3]
    self.genus = row[4]
    self.scientificName = row[5]
    if len(row[6]) > 0:
        self.associatedMedia = row[6].split("|")
    else:
        self.associatedMedia = []
    self.firstEvent = row[7]
    self.lastEvent = row[8]
    self.lastNumEvents = int(row[9])
    self.numEventsPresent = int(row[10])
    self.numIndividuals = int(row[11])

class Event:
  def __init__(self, row):
    self.ID = row[0]
    self.date = row[1]
    self.numNights = int(row[7])
    self.remarks = row[9]
    self.yearmonth = self.date[0:7]
    self.year = int(self.date[0:4])
    self.month = int(self.date[5:7])
    self.day = int(self.date[8:10])
    
# update_tables
#   Update the individuals and nights tables with the count for a particular taxon in a single occurrence record.
#   The associated event record provides the count of nights over which the individuals were collected. Counts
#   are pro-rated across months where required.

def update_tables(individuals, nights, event, taxon_index, count):
    time_index = (event.year - start_year) * 12 + event.month - 1
    if event.numNights > event.day:
        individuals[taxon_index][time_index] += count * (event.day / event.numNights)
        if time_index > 0:
            individuals[taxon_index][time_index - 1] += count * ((event.numNights - event.day) / event.numNights)
        nights[taxon_index][time_index] += event.numNights * (event.day / event.numNights)
        if time_index > 0:
            nights[taxon_index][time_index - 1] += event.numNights * ((event.numNights - event.day) / event.numNights)
    else:
        individuals[taxon_index][time_index] += count
        nights[taxon_index][time_index] += event.numNights
    
    
taxa = []
taxon_dict = {}
csv_filename = os.path.join(local_folder, "taxon.csv")
if os.path.isfile(csv_filename):
    with open(csv_filename, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headings = next(csv_reader)
        for row in csv_reader:
            taxon = Taxon(row)
            taxon_dict[taxon.ID] = len(taxa)
            taxa.append(taxon)

events = []
event_dict = {}
csv_filename = os.path.join(local_folder, "event.csv")
if os.path.isfile(csv_filename):
    with open(csv_filename, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headings = next(csv_reader)
        for row in csv_reader:
            event = Event(row)
            event_dict[event.ID] = len(events)
            events.append(event)
            
individuals = [ [0] * num_years * 12 for i in range(len(taxa)) ]
nights = [ [0] * num_years * 12 for i in range(len(taxa)) ]

csv_filename = os.path.join(local_folder, "occurrence.csv")
if os.path.isfile(csv_filename):
    with open(csv_filename, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headings = next(csv_reader)
        for row in csv_reader:
            event = events[event_dict[row[1]]]
            update_tables(individuals, nights, event, taxon_dict[row[2]], int(row[8]))

### Mean individuals per night

Use the data in the individuals and nights tables to generate two additional tables:

 * **monthly_means** - The average number of individuals of each taxon recorded per night of sampling within a given calendar month across all years
 * **annual_means** - The average number of individuals of each taxon recorded per night of sampling within a given year in which sampling occurred across all 12 months, with None as the value where sampling was incomplete.

In [8]:
mean_individuals = pd.DataFrame(individuals) / pd.DataFrame(nights)
value_present = mean_individuals.notna()

monthly_means = pd.DataFrame([ [None] * 12 for i in range(len(taxa)) ])
for t in range(len(taxa)):
    for m in range(12):
        years_counted = 0
        total_of_means = 0
        for y in range(num_years):
            i = y * 12 + m
            if value_present.iloc[t, i]:
                years_counted += 1
                total_of_means += mean_individuals.iloc[t, i]
        monthly_means.iloc[t, m] = total_of_means / years_counted

annual_means = pd.DataFrame([ [None] * num_years for i in range(len(taxa)) ])
for t in range(len(taxa)):
    for y in range(num_years):
        start_index = y * 12
        end_index = start_index + 12
        annual_means.iloc[t, y] = mean_individuals.iloc[t,start_index:end_index].sum(min_count=1) / 12
annual_means = annual_means.where(pd.notnull(annual_means), None)

### Taxon trends

Generate a grid comprising one row for the combination of all species that were consistently recorded throughout 1992 to 2019 and one row for each taxon monitored at any point in this period. Each row contains the following:

 * The verbatim identification provided for the species, with a slope representing change per year in the average number of recorded individuals per night per year (only when more than five years are available for comparison)
 * A representative image of the taxon, when available - these images show actual individuals collected during the period but may not represent the full range of species included under operation taxonomic unit names
 * Plot of mean number of recorded individuals per night for each year with adequate sampling, including a regression line and 95% confidence intervals in cases where more than five years are available for comparison
 * Histogram of mean number of recorded individuals per night of sampling in each calendar month across all years

In [29]:
def plot_years(x, y):
    for i in range(len(x) - 1, -1, -1):
        if y[i] is None:
            x.pop(i)
            y.pop(i)

    df = pd.DataFrame({'year': x,
                       'per night': y})
    df['year'] = df['year'].astype(float)
    X = sm.add_constant(df['year'].values)
    ols_model = sm.OLS(df['per night'].values, X)
    est = ols_model.fit()
    out = est.conf_int(alpha=0.05, cols=None)

    fig, ax = plt.subplots(figsize=(8, 2))
    df.plot(x='year',y='per night',linestyle='None',marker='s', ax=ax)
    
    if len(x) > 5:
        y_pred = est.predict(X)
        x_pred = df.year.values
        ax.plot(x_pred,y_pred)
        pred = est.get_prediction(X).summary_frame()
        ax.plot(x_pred,pred['mean_ci_lower'],linestyle='--',color='blue')
        ax.plot(x_pred,pred['mean_ci_upper'],linestyle='--',color='blue')
        slope = " (slope %.4f)" % (y_pred[1] - y_pred[0])
    else:
        slope = ""
    plt.xlim([1996.5, 2019.5])
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    plt.close()
    img_buf.seek(0)
    return img_buf.read(), slope

def plot_months(x, y):
    fg = plt.figure(figsize=(4, 2), dpi=80)
    ax = fg.gca()
    ax.set_xticks(np.arange(len(x)))
    ax.set_xticklabels(month_labels)
    plt.bar(x, y)
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    plt.close()
    img_buf.seek(0)
    return img_buf.read()

def get_widgets(text, image_name, year_buf, slope, month_buf):
    label = widgets.Label(text + slope)
    label.layout.object_position = "left center"
    if image_name is not None:
        image = PIL.Image.open(image_name)
        img_height = 160
        img_width = int((image.size[0] * img_height) / image.size[1])
        image.resize((img_width, img_height))
        img_buf = io.BytesIO()
        image.save(img_buf, format='png')
        img_buf.seek(0)
        image_widget = widgets.Image(value=img_buf.read(), format="png", width = img_width, height = img_height)
        image_widget.layout.object_position = "center center"
    else:
        image_widget = widgets.Label("")
    year_plot = widgets.Image(value=year_buf, format='png', width = 640, height = 160)
    year_plot.layout.object_position = "left center"
    month_plot = widgets.Image(value=month_buf, format='png', width = 320, height = 160)
    month_plot.layout.object_position = "left center"
    return [label, image_widget, year_plot, month_plot]
    
year_totals = [None] * 28
month_totals = [0] * 12
n = 0
startEvent = events[0].ID
endEvent = events[len(events) - 1].ID
value_present = annual_means.notna()
for t in range(len(taxa)):
    taxon = taxa[t]
    if taxon.firstEvent == startEvent and taxon.lastEvent == endEvent:
        n += 1
        for i in range(28):
            if value_present.iloc[t, i]:
                if year_totals[i] is None:
                    year_totals[i] = annual_means.iloc[t, i]
                else:
                    year_totals[i] += annual_means.iloc[t, i]
        for i in range(12):
            month_totals[i] += monthly_means.iloc[t, i]
year_buf, trend = plot_years(list(range(1992, 2020)), year_totals)
month_buf = plot_months(list(range(12)), month_totals)
plots = get_widgets("%d species" % n, None, year_buf, trend, month_buf)

for t in range(len(taxa)):
    x = list(range(1992, 2020))
    y = annual_means.iloc[t,:].tolist()
    year_buf, trend = plot_years(x, y)
    x = list(range(12))
    y = monthly_means.iloc[t,:].tolist()
    month_buf = plot_months(x, y)
    plots += get_widgets(taxa[t].verbatimIdentification, os.path.join(local_folder, taxa[t].associatedMedia[0]) if len(taxa[t].associatedMedia) > 0 else None, year_buf, trend, month_buf)
    
widgets.GridBox(plots, layout=widgets.Layout(grid_template_columns="15% 10% 50% 25%", align_items="center"))

GridBox(children=(Label(value='109 species (slope -0.2952)', layout=Layout(object_position='left center')), La…